In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('./Specialist Classification - Specialist Classification.csv')

In [ ]:
df_train = df.iloc[:, [3,5]]

In [ ]:
df_train.head()

,Problem In English,Specialist
0,I have a lot of headache for three days,Medicine
1,I have to hold my head as soon as I am in the ...,Medicine
2,My sister's eyes and headache for a few days,Medicine
3,My mom causes a lot of pain on the two sides o...,Medicine
4,I have a lot of headache,Medicine


In [ ]:
from keras.preprocessing.text import Tokenizer
import numpy as np

In [ ]:
df_train.iloc[:,1]

0             Medicine
1             Medicine
2             Medicine
3             Medicine
4             Medicine
            ...       
653            Surgery
654            Dentist
655               Skin
656            Surgery
657    Ophthalmologist
Name: Specialist, Length: 658, dtype: object

In [ ]:
target_tokenizer = Tokenizer(30)
target_tokenizer.fit_on_texts(df_train.iloc[:,1])
target_vocab_size = len(target_tokenizer.word_index) + 1
df_train.iloc[:,1] = np.array(target_tokenizer.texts_to_sequences(df_train.iloc[:,1])) -1
# df_train.iloc[:,1] = df_train.iloc[:,1].reshape(df_train.iloc[:,1].shape[0])
print("target_vocab_size:",target_vocab_size)

target_vocab_size: 10


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1951: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[selected_item_labels] = value


In [ ]:
target_tokenizer.word_index.items()

dict_items([('medicine', 1), ('ophthalmologist', 2), ('gynecologist', 3), ('surgery', 4), ('orthopedic', 5), ('cardiologist', 6), ('skin', 7), ('dentist', 8), ('ent', 9)])

In [ ]:
df_train.head()

,Problem In English,Specialist
0,I have a lot of headache for three days,0
1,I have to hold my head as soon as I am in the ...,0
2,My sister's eyes and headache for a few days,0
3,My mom causes a lot of pain on the two sides o...,0
4,I have a lot of headache,0


In [ ]:
target_reverse_word_index = {v: k for k, v in list(target_tokenizer.word_index.items())}
target_reverse_word_index

{1: 'medicine',
 2: 'ophthalmologist',
 3: 'gynecologist',
 4: 'surgery',
 5: 'orthopedic',
 6: 'cardiologist',
 7: 'skin',
 8: 'dentist',
 9: 'ent'}

In [ ]:
from tensorflow.keras.utils import to_categorical
y_train = to_categorical(df_train.Specialist)
# y_test = to_categorical(df_test.Sentiment)

In [ ]:
!pip install transformers
import transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 7.3 MB/s 
     |████████████████████████████████| 596 kB 63.1 MB/s 
     |████████████████████████████████| 101 kB 13.9 MB/s 
     |████████████████████████████████| 6.6 MB 45.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from transformers import AutoTokenizer,TFBertModel
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
bert = TFBertModel.from_pretrained('bert-base-cased')

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/502M [00:00<?, ?B/s]

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
# Tokenize the input (takes some time) 
# here tokenizer using from bert-base-cased
x_train = tokenizer(
    text=df_train.iloc[:,0].tolist(),
    add_special_tokens=True,
    max_length=45,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)

In [ ]:
input_ids = x_train['input_ids']
attention_mask = x_train['attention_mask']

In [ ]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Dense

In [ ]:
max_len = 45
input_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
input_mask = Input(shape=(max_len,), dtype=tf.int32, name="attention_mask")
embeddings = bert(input_ids,attention_mask = input_mask)[0] 
out = tf.keras.layers.GlobalMaxPool1D()(embeddings)
out = Dense(128, activation='relu')(out)
out = tf.keras.layers.Dropout(0.1)(out)
out = Dense(32,activation = 'relu')(out)
y = Dense(9,activation = 'sigmoid')(out)
model = tf.keras.Model(inputs=[input_ids, input_mask], outputs=y)
model.layers[2].trainable = True

In [ ]:
optimizer = Adam(
    learning_rate=5e-05, # this learning rate is for bert model , taken from huggingface website 
    epsilon=1e-08,
    decay=0.01,
    clipnorm=1.0)
# Set loss and metrics
loss =CategoricalCrossentropy(from_logits = True)
metric = CategoricalAccuracy('balanced_accuracy'),
# Compile the model
model.compile(
    optimizer = optimizer,
    loss = loss, 
    metrics = metric)

In [ ]:
train_history = model.fit(
    x ={'input_ids':x_train['input_ids'],'attention_mask':x_train['attention_mask']} ,
    y = y_train,
    validation_split = 0.2,
  epochs=20,
    batch_size=36
)

Epoch 1/20


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


15/15 [==============================] - 39s 545ms/step - loss: 1.8408 - balanced_accuracy: 0.4506 - val_loss: 2.6828 - val_balanced_accuracy: 0.0000e+00
Epoch 2/20
15/15 [==============================] - 5s 348ms/step - loss: 1.4406 - balanced_accuracy: 0.5361 - val_loss: 2.5107 - val_balanced_accuracy: 0.0606
Epoch 3/20
15/15 [==============================] - 5s 350ms/step - loss: 1.0654 - balanced_accuracy: 0.6673 - val_loss: 1.7551 - val_balanced_accuracy: 0.4015
Epoch 4/20
15/15 [==============================] - 5s 352ms/step - loss: 0.6613 - balanced_accuracy: 0.8156 - val_loss: 1.6038 - val_balanced_accuracy: 0.5076
Epoch 5/20
15/15 [==============================] - 5s 354ms/step - loss: 0.3745 - balanced_accuracy: 0.8992 - val_loss: 1.5033 - val_balanced_accuracy: 0.6288
Epoch 6/20
15/15 [==============================] - 5s 354ms/step - loss: 0.2263 - balanced_accuracy: 0.9430 - val_loss: 1.0256 - val_balanced_accuracy: 0.7576
Epoch 7/20
15/15 [============================

In [ ]:
# texts = input(str('I am having chest pain'))
texts = 'I had a bike accident yesterday, my knee is hurting'
x_val = tokenizer(
    text=texts,
    add_special_tokens=True,
    max_length=45,
    truncation=True,
    padding='max_length', 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True) 
prediction = model.predict({'input_ids':x_val['input_ids'],'attention_mask':x_val['attention_mask']})*100
# print(prediction[0].argsort()[::-1])
result= {}
for i in prediction[0].argsort()[::-1][:3]:
  result[target_reverse_word_index[i+1]] = prediction[0][i]



In [ ]:
result

{'cardiologist': 91.30915, 'medicine': 98.86409, 'orthopedic': 99.95426}

In [ ]:
# print(prediction[0].argsort()[::-1])
for i in prediction[0].argsort()[::-1][:3]:
  print(target_reverse_word_index[i+1], prediction[0][i])

orthopedic 99.95426
medicine 98.86409
cardiologist 91.30915


In [ ]:
target_reverse_word_index

{1: 'medicine',
 2: 'ophthalmologist',
 3: 'gynecologist',
 4: 'surgery',
 5: 'orthopedic',
 6: 'cardiologist',
 7: 'skin',
 8: 'dentist',
 9: 'ent'}

In [ ]:
!pip install pyngrok==4.1.1
!ngrok authtoken '2C37jp7qHi4ElwDfWwlfta4lcON_5Xi2uH6r2f1oo5BwDiA4L'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for pyngrok: filename=pyngrok-4.1.1-py3-none-any.whl size=15983 sha256=15366dae26910f7de4b48b47e0d5dda24e17e82c7738b6890b12cbd12b82647e
  Stored in directory: /root/.cache/pip/wheels/b1/d9/12/045a042fee3127dc40ba6f5df2798aa2df38c414bf533ca765
Successfully built pyngrok
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
!pip install flask-ngrok
!pip install flask==0.12.2  # Newer versions of flask don't work in Colab
                            # See https://github.com/plotly/dash/issues/257

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 83 kB 1.5 MB/s 
  Attempting uninstall: flask
    Found existing installation: Flask 1.1.4
    Uninstalling Flask-1.1.4:
      Successfully uninstalled Flask-1.1.4


**Markov Chain Prediction Code**

In [ ]:
!git clone https://github.com/gulraizchoudhary/predict_next_diagnosis_using_markovchain.git

def confusionMatrix(total_classes, gt, predicted):
    total = total_classes-len(gt)
    TP = len(list(set(predicted).intersection(set(gt))))
    FP = len(predicted)-TP
    FN = len(gt)-TP
    TN = total_classes-(TP+FP+FN)
    Accuracy = (TP+TN)/total
    MissclassificationRate = (FP+FN)/total
    TPR = TP/(FN+TP) #Sensitivity, recall
    FPR = FP/(TN+FP) 
    TNR= TN/(TN+FP) #Specificity, selectivity
    FNR= FN/(FN+TP) #miss rate
    Precision=0
    Fscore =0
    LikelihoodRatio=0
    
    if (FP+ TP) !=0:
        Precision = TP/(FP+ TP)
    
    AUC = (TP+(0.5*FP))/len(predicted)
    
    if (Precision+TPR) !=0:
        Fscore = 2*((Precision*TPR)/(Precision+TPR))
    
    if (1-TPR) !=0:
        LikelihoodRatio = Precision/(1-TPR)
    
    return TP, FP, FN, TN, Accuracy, MissclassificationRate, TPR, FPR, FNR, TNR, Precision, AUC, Fscore, LikelihoodRatio


def getCM(predicted, t_classes):
    cm =[]
    for list in predicted:
        if len(list[1])>0:
            cm.append(confusionMatrix(t_classes, list[0], list[1]))
    
    return cm

def printStat(cm):
    confusion_map=[list(i) for i in zip(*cm)]
    onfusion_mean =[np.mean(k) for k in confusion_map]
    onfusion_std =[np.std(k) for k in confusion_map]
    print("TP: "+str(onfusion_mean[0]))
    print("FP: "+str(onfusion_mean[1]))
    print("FN: "+str(onfusion_mean[2]))
    print("TN: "+str(onfusion_mean[3]))
    print("Accuracy: "+str(onfusion_mean[4]))
    print("MissclassificationRate: "+str(onfusion_mean[5]))
    print("TPR: "+str(onfusion_mean[6]))
    print("FPR: "+str(onfusion_mean[7]))
    print("FNR: "+str(onfusion_mean[8]))
    print("TNR: "+str(onfusion_mean[9]))
    print("Precision: "+str(onfusion_mean[10]))
    print("AUC: "+str(onfusion_mean[11]))
    print("AUC STD: "+str(onfusion_std[11]))
    print("Fscore: "+str(onfusion_mean[12]))
    print("LikelihoodRatio: "+str(onfusion_mean[13]))
    

import numpy as np
from sklearn.model_selection import train_test_split


# Markov chain stored as adjacency list.
markov = {}

def update_markov(current : str, next : str) -> None:
    """Add item to the markov.
    Args:
        current (three digit ICD-10 codes is str): Input ICD-10 code.
        next (ICD-10 code as str): Output ICD-10 code.
    """

    # Add the input ICD-10 code to the lexicon if it in there yet.
    if current not in markov:
        markov.update({current: {next: 1} })
        return

    # Retrieve the probabilties of the input ICD-10 code.
    options = markov[current]

    # Check if the output ICD-10 codes that is in the propability list.
    if next not in options:
        options.update({next : 1})
    else:
        options.update({next : options[next] + 1})

    # Update the markov
    markov[current] = options

def normalize() -> None:
    """normalize the frequencies to a 0-1 float"""
    for code, transition in markov.items():
        transition = dict((key, value / sum(transition.values())) for key, value in transition.items())
        markov[code] = transition
    

def predict(code : str) -> str:
    """Attempt to predict the next ICD-10 code in the markov chain.
    Args:
        ICD-10 code (str): Last ICD-10 known code from patient history.
    Returns:
        str: Next ICD-10 code.
        None: current ICD-10 is not in the markov chain.
    """
    if code not in markov:
        return None

    options = markov[code]
    print("markov options")
    print(options)

    # dict.keys()[-1]
    p = dict(sorted(options.items(), key=lambda item: item[1]))
    print(p)
    print(list(p)[-1])

    return list(p)[-1]
    

def train_markov(train):
    """update the markov chain using train"""
    
    for line in train:
        # Update markov chain.
        code = line.strip().split(' ')
        for i in range(len(code) - 1):
            update_markov(code[i], code[i+1])

def predict_next(test):
    """Predict the next ICD-10 code from a given"""
    pred = []
    gt = []
    for line in test:
        codes = line.strip().split(' ')
        print(codes)
        #Divide the sequence into two equals portion as a training and testing
        t_head, t_tail = codes[:len(codes)//2], codes[len(codes)//2:]
        print(t_head[-1])
        print("tail")
        print(t_tail)
    
        # Select the last ICD-10 code from the training sequence of the given set.
        p_code = predict(t_head[-1])
        print("p_code ".format(p_code))
        
        #book keeping the total number of the classes
        gt.append(t_tail[0])
        gt.append(p_code)
        
        pred.append((str(p_code),t_head[-1]))
        
    
    return pred, len(set(gt))


def load_dataset():
    """1- Load the dataset from the file
       2- Split dataset into train and test
    """
    data = tuple(open("./predict_next_diagnosis_using_markovchain/dataset.txt", 'r'))
    
    #split data into training and testing    
    train, test = train_test_split(data,test_size=0.2)
    
    return train, test


#Split data into train and test
train, test = load_dataset()

#update the markov model
train_markov(train)


#Normalize the markov model
normalize()



Cloning into 'predict_next_diagnosis_using_markovchain'...
remote: Enumerating objects: 22, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 22 (delta 5), reused 13 (delta 2), pack-reused 0
Unpacking objects: 100% (22/22), done.


In [ ]:
!pip install icd10-cm
data = tuple(open("./predict_next_diagnosis_using_markovchain/dataset.txt", 'r'))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 675 kB 7.0 MB/s 


In [ ]:
import icd10
unique_codes = set()
for line in data:
  for code in line.strip().split(' '):
    unique_codes.add(code)

icd10_code_list = []
for code in unique_codes:
  try:
    icd10_code_list.append(code + ": " + icd10.find(code).description)
  except:
    print('Error in ' + code)

print(icd10_code_list)
print(type(icd10_code_list))

Error in T08
Error in T90
Error in F00
['S30: Superficial injury of abdomen, lower back, pelvis and external genitals', 'D65: Disseminated intravascular coagulation [defibrination syndrome]', 'O10: Pre-existing hypertension complicating pregnancy, childbirth and the puerperium', 'O30: Multiple gestation', 'M60: Myositis', 'O20: Hemorrhage in early pregnancy', 'I26: Pulmonary embolism', 'E10: Type 1 diabetes mellitus', 'C45: Mesothelioma', 'F40: Phobic anxiety disorders', 'M86: Osteomyelitis', 'E65: Localized adiposity', 'T20: Burn and corrosion of head, face, and neck', 'M30: Polyarteritis nodosa and related conditions', 'M65: Synovitis and tenosynovitis', 'A00: Cholera', 'C81: Hodgkin lymphoma', 'B85: Pediculosis and phthiriasis', 'C76: Malignant neoplasm of other and ill-defined sites', 'C60: Malignant neoplasm of penis', 'G60: Hereditary and idiopathic neuropathy', 'H55: Nystagmus and other irregular eye movements', 'J90: Pleural effusion, not elsewhere classified', 'M05: Rheumatoid

**Firebase APIS**

In [ ]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

# Use a service account
cred = credentials.Certificate("./uhi-hackathon-firebase-adminsdk-rloxw-52f80c9011.json")
firebase_admin.initialize_app(cred)

db = firestore.client()

**Flask Server**

In [ ]:
# flask_ngrok_example.py
from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok
import json
import random
import string
import datetime

app = Flask(__name__)
app.config['JSONIFY_PRETTYPRINT_REGULAR'] = False
run_with_ngrok(app)  # Start ngrok when app is run

@app.route("/predict")
def medical_triage_prediction():
    # print(request.args.get("text"))
    input_text = request.args.get("text")

    print(input_text)

    x_val = tokenizer(
      text=input_text,
      add_special_tokens=True,
      max_length=45,
      truncation=True,
      padding='max_length', 
      return_tensors='tf',
      return_token_type_ids = False,
      return_attention_mask = True,
      verbose = True) 
    prediction = model.predict({'input_ids':x_val['input_ids'],'attention_mask':x_val['attention_mask']})*100

    result= []

    for i in prediction[0].argsort()[::-1][:3]:
      result.append(target_reverse_word_index[i+1].capitalize())

    print(result)
    print(type(result))
    # print(jsonify(result))

    return jsonify(first=result[0], second=result[1], third=result[2])

@app.route("/icd10-codes")
def get_icd10_code_list():
    input_text = request.args.get("text")


    return jsonify(icd10_code_list)

@app.route("/predict-icd10-codes")
def predict_icd10_code():
    input_code = request.args.get("text").split(":")[0]
    p_code = predict(input_code)
    return p_code + " : " + icd10.find(p_code).description

@app.route("/symptoms")
def get_symptoms():

    users_ref = db.collection(u'Symptoms')
    docs = users_ref.stream()

    data = []
    for doc in docs:
        data.append(doc.to_dict())

    return jsonify(data)

@app.route("/latest-symptoms")
def get_patients_symptoms():
    data_in_db = db.collection(u'Patients-Symptoms-Datastore').order_by('timestamp', direction=firestore.Query.DESCENDING).limit(1).stream()
    data = []
    for b in data_in_db:
        data.append(b.to_dict())
    return jsonify(data[0]['symptoms'])  

@app.route("/save-symptoms", methods=['POST'])
def save_patients_symptoms():
    input_json = request.get_json(force=True)
    symptoms_list = [input_json['symptom']]
    symptoms_list.extend(input_json['questions'])
    symptoms_dict = {
        "timestamp":datetime.datetime.now(),
        "symptoms": symptoms_list
    }

    db.collection(u'Patients-Symptoms-Datastore').document(''.join(random.choices(string.ascii_uppercase + string.digits, k=100))).set(symptoms_dict)
    return "Success"

                                                    

if __name__ == '__main__':
    app.run()  # If address is in use, may need to terminate other sessions:
               # Runtime > Manage Sessions > Terminate Other Sessions

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://e1f5-34-83-179-179.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


**Code to populate Firebase Symptoms Questions and Answers Database**



In [ ]:
import json
complete_data = json.load(open('./symptom_checker.json'))
count = 1
for data in complete_data:
  # print(data)
  db.collection(u'Symptoms').document(str(count)).set(data)
  count = count+1
